In [1]:
pwd

'C:\\Users\\Edgard_Cuadra\\OneDrive\\Desktop\\Ironhack\\Projects\\Project_2_Pipelines'

In [2]:
import src.clean as cl

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re 

### Extracting Experts players rankings from CBS sports 

First we find the webpage we want to get the information from so that we can scrape the data out of it. to make it legible we transform in into html format by using beutifulsoup.

In [4]:
urlcbs = "https://www.cbssports.com/fantasy/football/rankings/"

In [5]:
cbshtml = cl.get_html(urlcbs)

once we get the content in a format we can use we can proceedd to extract the information wanted, in this cases we need player name, team, position and ranking.

In [9]:
player1 = cl.select_html(cbshtml,"span.player-name")

In [12]:
player1_list = [element.getText().strip() for element in player1]

In [15]:
ranking1 = cl.select_html(cbshtml,"div.rank")

In [16]:
ranking1_list = [element.getText().strip() for element in ranking1]

In [27]:
team1 = cl.select_html(cbshtml,"span.team.position")

In [28]:
team1_list = [element.getText().strip() for element in team1]

In [37]:
Team1_ = [element.split("\n")[0].strip(",").strip()  for element in team1_list]

In [39]:
position1_ = [element.split("\n")[1].strip(",").strip()  for element in team1_list]

Now that we have extracted the information into separate lists we need to converge them into a dictionary in order for us to be able to turn it into a dataframe.

In [53]:
allRanking ={
    "Player": player1_list,
    "Position": position1_,
    "Team": Team1_,
    "Ranking" : ranking1_list,
}

In [56]:
allrankings = pd.DataFrame(allRanking)
allrankings.head(2)

,Player,Position,Team,Ranking
0,J. Taylor,RB,IND,1
1,C. McCaffrey,RB,CAR,2


the web page pulled all the rankings into one dataframe, we need proceed to great 3 dataframes with the rankings. using iloc we are going to cut the dataframe into 3 tables and after merge them with the name index.

In [163]:
eisenbergranking = allrankings.iloc[0:200,:].reset_index(drop=True)
richardranking = allrankings.iloc[200:400,:].reset_index(drop=True)
cummingsranking = allrankings.iloc[400:600,:].reset_index(drop=True)

In [70]:
from functools import reduce

In [183]:
x = [eisenbergranking, richardranking, cummingsranking]

test = reduce(lambda left,right: pd.merge(left,
right,on=['Player',"Position","Team"],how='outer'), x)

In [187]:
test.sample(3)

,Player,Position,Team,Ranking_x,Ranking_y,Ranking
41,T. McLaurin,WR,WAS,42,45,41
91,A. St. Brown,WR,DET,92,91,71
33,K. Pitts,TE,ATL,34,37,57


since we want the top 100 players to look out after, we need to drop the rows bellow line 100 and use the information of the top ranked prospects in the league.

In [215]:
top100ranked = test.head(120)

we check if there is any missing values of information of the collumns that we need to make sure we have the required information for all the players in the top 100, and since we have to make a calculated column we need to change the type of some columns to integers

In [216]:
cl.columnnullcount(top100ranked)

Player       0
Position     0
Team         0
Ranking_x    0
Ranking_y    0
Ranking      0
dtype: int64

In [217]:
top100ranked["Ranking_x"] = top100ranked["Ranking_x"].apply(int)

C:\Users\Edgard_Cuadra\AppData\Local\Temp\ipykernel_23388\2618516453.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top100ranked["Ranking_x"] = top100ranked["Ranking_x"].apply(int)


In [218]:
top100ranked["Ranking_y"] = top100ranked["Ranking_y"].apply(int)

C:\Users\Edgard_Cuadra\AppData\Local\Temp\ipykernel_23388\230294182.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top100ranked["Ranking_y"] = top100ranked["Ranking_y"].apply(int)


In [219]:
top100ranked["Ranking"] = top100ranked["Ranking"].apply(int)

C:\Users\Edgard_Cuadra\AppData\Local\Temp\ipykernel_23388\1780496477.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top100ranked["Ranking"] = top100ranked["Ranking"].apply(int)


In [220]:
top100ranked.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120 entries, 0 to 119
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Player     120 non-null    object
 1   Position   120 non-null    object
 2   Team       120 non-null    object
 3   Ranking_x  120 non-null    int64 
 4   Ranking_y  120 non-null    int64 
 5   Ranking    120 non-null    int64 
dtypes: int64(3), object(3)
memory usage: 6.6+ KB


now we need to creat a new column with an average agregate of the ranking for every player to get our ranking value as an average of all three experts rankings

In [221]:
top100ranked['My_Ranking'] = (top100ranked.Ranking_x + top100ranked.Ranking_y + top100ranked.Ranking)/3

C:\Users\Edgard_Cuadra\AppData\Local\Temp\ipykernel_23388\748878491.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top100ranked['My_Ranking'] = (top100ranked.Ranking_x + top100ranked.Ranking_y + top100ranked.Ranking)/3


In [222]:
top100ranked.head(2)

,Player,Position,Team,Ranking_x,Ranking_y,Ranking,My_Ranking
0,J. Taylor,RB,IND,1,1,1,1.000000
1,C. McCaffrey,RB,CAR,2,5,3,3.333333


now that we have the desired outcome i need to change the columns name, and eliminate columns that are not usefull or repetitive. after that we can rearrange by the column of my ranking and reset the index with that order.

In [223]:
top100ranked = top100ranked.rename(columns={"Ranking_x": "Eisenberg_ranking","Ranking_y": "Richards_ranking", "Ranking": "Cummings_ranking"})

In [224]:
top100ranked = top100ranked.round(2) #this will roun to 2 decimal places

In [225]:
top100ranked.sample(3)

,Player,Position,Team,Eisenberg_ranking,Richards_ranking,Cummings_ranking,My_Ranking
63,J. Smith-Schuster,WR,KC,64,78,63,68.33
67,C. Edmonds,RB,MIA,68,71,68,69.00
0,J. Taylor,RB,IND,1,1,1,1.00


In [226]:
top100ranked = top100ranked.sort_values(by=["My_Ranking"]).reset_index(drop=True)

In [227]:
top100ranked.sample(3)

,Player,Position,Team,Eisenberg_ranking,Richards_ranking,Cummings_ranking,My_Ranking
107,M. Carter,RB,NYJ,117,109,94,106.67
115,C. Watson,WR,GB,119,130,120,123.00
92,E. Moore,WR,NYJ,91,100,81,90.67


### Importing players stats and salary.

the database that is being impor is the general starts of all the NFL players and their official salary. 

In [230]:
pwd

'C:\\Users\\Edgard_Cuadra\\OneDrive\\Desktop\\Ironhack\\Projects\\Project_2_Pipelines'

In [231]:
nfl_stat = pd.read_csv('.\\Input\\madden21_ratings.csv', encoding='latin1')

the table we imported has too much information that we dont need so we will drop the columns that are of no use to us, we can use the following code to eliminate by column position and make it easier and faster for us to drop the columns.

In [234]:
nfl_stat2 = nfl_stat.drop(nfl_stat.columns[[11,12,13,14,15,16,17,20,21,22,23,24,25,26,28,29,30,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,58,62,63]], axis=1) 

In [235]:
nfl_stat2.head(3)

,Team,Full Name,Overall Rating,Position,Age,Speed,Acceleration,Awareness,Agility,Strength,...,Tackle,Injury,Jersey Number,Total Salary,Signing Bonus,Years Pro,Height,Weight,Birthdate,College
0,Rams,Aaron Donald,99,RE,29,82,90,99,86,99,...,92,98,99,"$101,892,000.00","$40,000,000.00",6,73,280,5/23/1991,Pittsburgh
1,Patriots,Stephon Gilmore,99,CB,29,92,94,99,94,70,...,68,94,24,"$33,550,000.00","$31,450,000.00",8,73,202,9/19/1990,South Carolina
2,Panthers,Christian McCaffrey,99,HB,24,92,93,97,97,72,...,36,96,22,"$45,840,000.00","$32,190,000.00",3,71,205,6/7/1996,Stanford


in order to merge the two tables by thei identifying names we will have to use the same numenclatures, thus the best approach would be to break into two columns and rearrange to get the desired initial and last name format.

In [242]:
nfl_stat2['lastname'] = nfl_stat2['Full Name'].str.split(' ', expand=True)[1]

In [252]:
nfl_stat2['initials_'] =nfl_stat2['Full Name'].str.split('', expand=True)[1]

In [254]:
nfl_stat2['short_name'] =  nfl_stat2['initials_']+ '. ' + nfl_stat2['lastname']

now we need to start to eliminate the columns that we created temporarly to have a clean table, with only the information that we need.

In [266]:
nfl_stat2 = nfl_stat2.drop(nfl_stat2.columns[[25,26,27]], axis=1)

In [268]:
nfl_stat2.sample(2)

,Team,Full Name,Overall Rating,Position,Age,Speed,Acceleration,Awareness,Agility,Strength,...,Injury,Jersey Number,Total Salary,Signing Bonus,Years Pro,Height,Weight,Birthdate,College,short_name
1815,Lions,Kevin Strong,62,DT,24,64,78,55,58,80,...,84,92,"$1,760,000.00",$-,1,76,285,8/5/1996,UTSA,K. Strong
800,Bills,Isaiah McKenzie,72,WR,25,92,93,70,94,53,...,87,19,"$680,000.00","$280,000.00",3,68,173,4/9/1995,Georgia,I. McKenzie


In [275]:
nfl_stat2 = nfl_stat2.rename(columns={"short_name": "Player"})

In [283]:
nfl_stat2.sample(5)

,Team,Full Name,Overall Rating,Position,Age,Speed,Acceleration,Awareness,Agility,Strength,...,Injury,Jersey Number,Total Salary,Signing Bonus,Years Pro,Height,Weight,Birthdate,College,Player
1524,Cardinals,Eno Benjamin,65,HB,21,87,88,63,85,67,...,88,26,"$2,737,714.00","$100,000.00",0,69,207,4/13/1999,Arizona St.,E. Benjamin
446,Seahawks,Rashaad Penny,78,HB,24,90,90,78,87,71,...,90,20,"$4,860,000.00","$5,920,000.00",2,71,220,2/2/1996,San Diego St.,R. Penny
980,Cowboys,Neville Gallimore,70,DT,23,79,84,69,74,83,...,92,75,"$4,229,084.00","$960,000.00",0,74,304,1/17/1997,Oklahoma,N. Gallimore
2011,Panthers,Chris Reed,59,C,28,70,81,70,53,85,...,84,64,"$2,380,000.00",$-,5,76,320,7/22/1992,Minnesota State,C. Reed
1164,Panthers,Keith Kirkwood,68,WR,25,91,87,71,85,62,...,87,19,"$750,000.00","$50,000.00",2,75,210,12/26/1994,Temple,K. Kirkwood


in order to merge both tables we need to tie the name with the nick name so we proceed to impor another table that will help us get the missing piece of the puzzel and complete the tables before me merge them.

In [306]:
teamlist = pd.read_csv('.\\Input\\Teamslist.csv', encoding='latin1')

In [329]:
p = [nfl_stat2, teamlist]

final = reduce(lambda left,right: pd.merge(left,
right,on=['Team'],how='outer'), p)

In [330]:
final.head(2)

,Team,Full Name,Overall Rating,Position,Age,Speed,Acceleration,Awareness,Agility,Strength,...,Jersey Number,Total Salary,Signing Bonus,Years Pro,Height,Weight,Birthdate,College,Player,ï»¿city
0,Rams,Aaron Donald,99,RE,29,82,90,99,86,99,...,99,"$101,892,000.00","$40,000,000.00",6,73,280,5/23/1991,Pittsburgh,A. Donald,LAR
1,Rams,Jalen Ramsey,94,CB,25,91,93,94,90,69,...,20,"$18,730,000.00","$13,250,000.00",4,73,208,10/24/1994,Florida State,J. Ramsey,LAR


Now that we have the table renamed we can proceed to change some colum names so they match when mergin.

In [331]:
final = final.rename(columns={"Team": "Nickname"})

In [332]:
final = final.rename(columns={"ï»¿city": "Team"})

In [333]:
final

,Nickname,Full Name,Overall Rating,Position,Age,Speed,Acceleration,Awareness,Agility,Strength,...,Jersey Number,Total Salary,Signing Bonus,Years Pro,Height,Weight,Birthdate,College,Player,Team
0,Rams,Aaron Donald,99,RE,29,82,90,99,86,99,...,99,"$101,892,000.00","$40,000,000.00",6,73,280,5/23/1991,Pittsburgh,A. Donald,LAR
1,Rams,Jalen Ramsey,94,CB,25,91,93,94,90,69,...,20,"$18,730,000.00","$13,250,000.00",4,73,208,10/24/1994,Florida State,J. Ramsey,LAR
2,Rams,Cooper Kupp,89,WR,27,88,91,93,95,65,...,10,"$4,070,000.00","$950,000.00",3,74,208,6/15/1993,Eastern Wash.,C. Kupp,LAR
3,Rams,Robert Woods,86,WR,28,90,88,90,87,63,...,17,"$31,000,000.00","$8,020,000.00",7,72,193,4/10/1992,USC,R. Woods,LAR
4,Rams,Johnny Hekker,86,P,30,69,79,72,64,56,...,6,"$16,560,000.00","$4,710,000.00",8,77,241,2/8/1990,Oregon State,J. Hekker,LAR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2288,Lions,Beau Benzschawel,58,C,24,64,77,69,68,82,...,63,"$1,760,000.00",$-,1,78,300,9/10/1995,Wisconsin,B. Benzschawel,DET
2289,Lions,John Atkins,58,DT,27,60,73,55,54,81,...,99,"$590,000.00",$-,2,75,320,12/21/1992,Georgia,J. Atkins,DET
2290,Lions,David Blough,56,QB,25,75,86,64,76,58,...,10,"$1,760,000.00",$-,1,72,200,7/31/1995,Purdue,D. Blough,DET
2291,Lions,Dan Skipper,54,RT,25,58,75,54,64,81,...,70,"$670,000.00",$-,3,81,325,9/20/1994,Arkansas,D. Skipper,DET


In [339]:
query = final [(final["Player"] =="C. Kupp")]

In [340]:
query

,Nickname,Full Name,Overall Rating,Position,Age,Speed,Acceleration,Awareness,Agility,Strength,...,Jersey Number,Total Salary,Signing Bonus,Years Pro,Height,Weight,Birthdate,College,Player,Team
2,Rams,Cooper Kupp,89,WR,27,88,91,93,95,65,...,10,"$4,070,000.00","$950,000.00",3,74,208,6/15/1993,Eastern Wash.,C. Kupp,LAR
